In [1]:
from importlib import reload
import fds_tools
reload(fds_tools)
import common_tools
reload(common_tools)
import db_tools
reload(db_tools)
import duckdb
reload(duckdb)
import json
import numpy as np
import pandas as pd
from db_tools.mySQL_tools import Connection, ConnectionConfigType, MySQLConfig
from fds_tools.fake_users_generator import FakeUsersGenerator, Constants
from fds_tools.data_cleaner import FFF
from common_tools import CommonTools, PandasTools, DuckDBTools
import duckdb
from collections import Counter
from cda_tools.location_cleaner import LocationCleaner

In [2]:
df_anime_azathoth_cleaned = DuckDBTools.load_path('data/anime_azathoth42/anime_cleaned.csv')
df_user_azathoth_cleaned = DuckDBTools.load_path('data/anime_azathoth42/users_cleaned.csv')
df_user_anime_azathoth_cleaned = DuckDBTools.load_path('data/anime_azathoth42/animelists_cleaned.csv')

df_anime_hernen_cleaned = DuckDBTools.load_path('data/anime_hernan4444/anime.csv')
df_anime_hernen_cleaned_syn = DuckDBTools.load_path('data/anime_hernan4444/anime_with_synopsis.csv')

In [3]:
duckdb.query("""select * from users_cleaned where username == 'gigguk'""")

┌──────────┬─────────┬───────────────┬────────────────┬─────────────┬──────────────┬──────────────────┬──────────────────────────┬─────────┬───────────────────┬─────────────────────┬─────────────┬─────────────────────┬─────────────────────┬──────────────────┬─────────────────┬────────────────┐
│ username │ user_id │ user_watching │ user_completed │ user_onhold │ user_dropped │ user_plantowatch │ user_days_spent_watching │ gender  │     location      │     birth_date      │ access_rank │      join_date      │     last_online     │ stats_mean_score │ stats_rewatched │ stats_episodes │
│ varchar  │  int64  │     int64     │     int64      │    int64    │    int64     │      int64       │          double          │ varchar │      varchar      │      timestamp      │   varchar   │      timestamp      │      timestamp      │      double      │     double      │     int64      │
├──────────┼─────────┼───────────────┼────────────────┼─────────────┼──────────────┼──────────────────┼────────────

In [ ]:
cleaner = LocationCleaner()
df = pd.read_csv("data/anime_azathoth42/users_cleaned.csv")
df["country"] = df["location"].apply(cleaner.clean)
df = cleaner.map_as_unknown(df, column="country", min_count=25)
df.to_csv("data/data_cda/users_cleaned.csv")

In [ ]:
duckdb.query(f"""
             CREATE OR REPLACE
             VIEW anime_nodes AS
             SELECT anime_id, title_english, score, members
             FROM read_csv_auto('data/anime_azathoth42/anime_cleaned.csv')
             WHERE members > 100_000 and score > 8
             """)
print(f"CUT: {duckdb.query("""select count(*) from anime_nodes""").fetchone()[0]}\tanime titles")

duckdb.query(f"""
             CREATE OR REPLACE
             VIEW user_nodes AS
             SELECT username, gender, country
             FROM read_csv_auto('data/data_cda/users_cleaned.csv')
             WHERE user_completed > 400
             """)

print(f"CUT: {duckdb.query("""select count(*) from user_nodes""").fetchone()[0]}\tusers")

duckdb.query(f"""CREATE OR REPLACE VIEW votes_cleaned AS
             SELECT v.*
             FROM read_csv_auto('data/anime_azathoth42/anime_cleaned.csv')
             join user_nodes u
             on v.username = u.username""")

# duckdb.query("""
#     SELECT
#         a1.anime_id AS source,
#         a2.anime_id AS target,
#         COUNT(*) AS weight
#     FROM animelists_random a1
#     JOIN animelists_random a2
#         ON a1.username = a2.username
#        AND a1.anime_id < a2.anime_id
#     GROUP BY source, target
#     HAVING weight > 0
#              """)

CUT: 313	anime titles
CUT: 13509	users


┌────────────────┬─────────┬───────────────┬────────────────┬─────────────┬──────────────┬──────────────────┬──────────────────────────┬─────────┬────────────────────────┬─────────────────────┬─────────────┬─────────────────────┬─────────────────────┬──────────────────┬─────────────────┬────────────────┐
│    username    │ user_id │ user_watching │ user_completed │ user_onhold │ user_dropped │ user_plantowatch │ user_days_spent_watching │ gender  │        location        │     birth_date      │ access_rank │      join_date      │     last_online     │ stats_mean_score │ stats_rewatched │ stats_episodes │
│    varchar     │  int64  │     int64     │     int64      │    int64    │    int64     │      int64       │          double          │ varchar │        varchar         │      timestamp      │   varchar   │      timestamp      │      timestamp      │      double      │     double      │     int64      │
├────────────────┼─────────┼───────────────┼────────────────┼─────────────┼───────

In [4]:
anime_name_list = duckdb.query("""select title_english, score, members from anime_cleaned""").fetchall()
anime_name_list = [(*i, 'anime') for i in anime_name_list if i[0] is not None]
anime_name_set = set(anime_name_list)
# assert len(anime_name_list) == len(anime_name_set) !!!!!!!!!

user_name_list = duckdb.query("""select username from users_cleaned""").fetchall()
user_name_list = [(*i, 1, 1, 1, 'user') for i in user_name_list if i[0] is not None]
user_name_set = set(user_name_list)
assert len(user_name_list) == len(user_name_set)

nodes_set = set(anime_name_set)
nodes_set.update(user_name_set)
# assert len(nodes_set) == len(anime_name_list) + len(user_name_list) !!!!!!!!!

In [5]:
con = duckdb.connect()
con.close()

In [4]:
duckdb.query("""
    select
        anime_id AS id,
        title_english AS name,
        score,
        members
    from anime_cleaned
    where score > 8 and title_english is not NULL 
""").to_csv("anime_nodes.csv", header=False)

In [5]:
duckdb.query("""
    select
        username AS id,
        gender
    FROM users_cleaned
    where user_completed > 500
""").to_csv("user_nodes.csv", header=False)

In [7]:

duckdb.query(f"""CREATE OR REPLACE VIEW anime_nodes AS SELECT * FROM read_csv_auto('anime_nodes.csv')""")
duckdb.query(f"""CREATE OR REPLACE VIEW user_nodes AS SELECT * FROM read_csv_auto('user_nodes.csv')""")
duckdb.query(f"""CREATE OR REPLACE VIEW votes AS SELECT * FROM read_csv_auto('data/anime_azathoth42/animelists_cleaned.csv')""")
duckdb.query(f"""CREATE OR REPLACE VIEW votes_cleaned AS
             SELECT v.*
             FROM votes v
             join user_nodes u
             on v.username = u.username""")

# duckdb.query("""
#     SELECT
#         a1.anime_id AS source,
#         a2.anime_id AS target,
#         COUNT(*) AS weight
#     FROM animelists_random a1
#     JOIN animelists_random a2
#         ON a1.username = a2.username
#        AND a1.anime_id < a2.anime_id
#     GROUP BY source, target
#     HAVING weight > 0
#              """)

BinderException: Binder Error: Values list "u" does not have a column named "username"

LINE 5:              on v.username = u.username
                                     ^